In [2]:
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import os
import random
import locale
from datetime import datetime
import tkinter as tk
from tkinter import ttk
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function

In [3]:
# Variable

path_dossier = "C:\\Users\\Alex\\Desktop\\Scrapping_papa\\data"


In [4]:
# Fonction pour variable

def creer_dossier_course():
    """
    Dans cette fonction je créer un dossier pour les courses qu'on sort aujourd'hui

    à vérifier si necessaire
    """
    date_actuelle = datetime.now().strftime("%Y-%m-%d")
    nom_dossier = f"scrapping_{date_actuelle}"
    chemin_dossier = os.path.join(path_dossier, nom_dossier)
    try:
        os.mkdir(chemin_dossier)
        print(f"Dossier créé : {nom_dossier}")
    except FileExistsError:
        print(f"Le dossier {nom_dossier} existe déjà.")


date_du_jour_global = None

def date_today():
    global date_du_jour_global
    if date_du_jour_global is None:  
        locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')  
        date_du_jour = datetime.now().strftime("%A %d %B %Y")
        date_du_jour_global = ' '.join([word.capitalize() for word in date_du_jour.split()])
    return date_du_jour_global

In [5]:
def show_popup():
    # Fonction pour supprimer les courses sélectionnées
    def delete_selected():
        selected_courses = [key for key, var in check_vars.items() if var.get()]
        for course in selected_courses:
            del links_dict[course]
        # Ferme la fenêtre une fois terminé
        popup.destroy()
        # Affiche le nouveau dictionnaire
        print("Nouveau dictionnaire :", links_dict)

    # Création de la fenêtre pop-up
    popup = tk.Toplevel(root)
    popup.title("Supprimer des courses")
    popup.geometry("400x400")

    # Variables pour les cases à cocher
    check_vars = {}
    for course in links_dict.keys():
        var = tk.BooleanVar(value=False)
        check_vars[course] = var
        cb = ttk.Checkbutton(popup, text=course, variable=var)
        cb.pack(anchor='w', padx=20, pady=5)

    # Bouton pour confirmer la suppression
    delete_button = ttk.Button(popup, text="Supprimer", command=delete_selected)
    delete_button.pack(pady=20)

In [6]:
# Fonction scrapping

def accept_coockies(driver:webdriver.Chrome):
    """ 
    Dans cette fonction je mets un temps au pire entre 5 et 8 secondes, j'attends l'affichage
    du bouton coockie et je click dessus.
    """
    time.sleep(random.uniform(5,8))
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "sd-cmp-1bquj"))).click()
    return driver

def extract_links(driver):
    """
    Fonction qui extrait les liens de différentes sections d'un tableau HTML.
    
    Args:
        driver: Instance de Selenium WebDriver.

    Returns:
        dict: Un dictionnaire contenant le texte des liens comme clés et les URLs comme valeurs.
    """
    links_dict = {}

    # Liste des sélecteurs CSS pour les différentes sections
    sections = [
        "tr.trTodayFirst td.tLeft.hideMobile",
        "tr.trTodayMiddle td.tLeft.hideMobile",
        "tr.trTodayLast td.tLeft.hideMobile"
    ]

    time.sleep(random.uniform(3, 5))

    for section in sections:
        # Trouver les éléments correspondant à la section
        titles = driver.find_elements(By.CSS_SELECTOR, section)

        for title in titles:
            try:
                # Récupérer le lien et son texte
                link = title.find_element(By.TAG_NAME, "a")
                text = link.text
                href = link.get_attribute("href")

                if text and href:
                    links_dict[text] = href
            except Exception as e:
                print(f"Erreur lors de l'extraction d'un lien : {e}")

        # Pause aléatoire entre 3 et 5 secondes
        time.sleep(random.uniform(3, 5))

    return links_dict

def scrape_links_to_dataframes(driver, links_dict, save_csv=False, show_preview=False):
    """
    Fonction qui scrape les tableaux HTML à partir d'une liste de liens via Selenium,
    les convertit en DataFrames, et les sauvegarde en CSV si demandé.

    Args:
        driver (WebDriver): Instance Selenium WebDriver.
        links_dict (dict): Dictionnaire contenant les noms des courses comme clés et leurs URLs comme valeurs.
        save_csv (bool): Si True, les DataFrames sont sauvegardés en fichiers CSV. Par défaut False.
        show_preview (bool): Si True, affiche un aperçu des DataFrames. Par défaut False.

    Returns:
        list: Liste de tuples contenant (nom de la course, index du tableau, DataFrame).
    """
    # Liste pour stocker les DataFrames
    all_dataframes = []

    for course_name, url in links_dict.items():
        print(f"Scraping: {course_name} ({url})")
        
        try:
            # Charger la page avec Selenium
            driver.get(url)
            time.sleep(random.uniform(1, 2))


            # Récupérer le HTML de la page
            html = driver.page_source
        except Exception as e:
            print(f"Erreur lors du chargement de la page : {url} - {e}")
            continue

        # Analyser le HTML avec BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')
        
        # Trouver tous les tableaux avec la classe 'tableauLine'
        tables = soup.find_all('table', class_='tableauLine')
        
        # Parcourir chaque tableau trouvé
        for table_idx, table in enumerate(tables):
            # Liste pour stocker les données
            data = []
            headers = []

            # Récupérer les en-têtes de colonnes
            header_row = table.find('thead').find_all('th')
            headers = [th.text.strip() for th in header_row]

            # Récupérer les lignes de données
            rows = table.find('tbody').find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                data.append([cell.text.strip() for cell in cells])
            
            # Créer un DataFrame pour ce tableau
            df = pd.DataFrame(data, columns=headers)
            
            # Ajouter le DataFrame à la liste
            all_dataframes.append((course_name, table_idx, df))

            # Afficher un aperçu pour vérifier
            if show_preview:
                print(f"Tableau {table_idx + 1} de {course_name} :")
                print(df.head())

            # Sauvegarder le DataFrame en CSV si demandé
            if save_csv:
                file_name = f"{course_name.replace(' ', '_')}_table{table_idx + 1}.csv"
                df.to_csv(file_name, index=False)
                print(f"Tableau sauvegardé : {file_name}")

    return all_dataframes

In [8]:
# Fonction obsolète

def close_ad(driver):
    """
    Ferme la publicité en cliquant sur le bouton de fermeture.
    """
    try:
        close_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div.close_buttons"))
        )
        close_button.click()
        print("Publicité fermée.")
    except Exception as e:
        print(f"Impossible de fermer la publicité : {e}")

In [87]:

def transformer_df(df):
    df1 = df[["N°", "Cheval", 'S/A', "Gain", "Dernières performances"]]

    # Traitement de la colonne 'Gain'
    df1['Gain'] = df1['Gain'].str.replace('€', '').str.replace(' ', '')
    df1['Gain'] = df1['Gain'].str.replace('\xa0', '', regex=True)
    #df1['Gain'] = df1['Gain'].astype(float) 
    #df1['Gain'] = (df1['Gain'] / 1000).round().astype(int)

    # Traitement de la colonne 'S/A'
    df1['S/A'] = df1['S/A'].str.replace('^H', '', regex=True)
    df1['S/A'] = df1['S/A'].str.replace('^M', '', regex=True)
    df1['S/A'] = df1['S/A'].str.replace('^F', '', regex=True)
    #df1['S/A'] = df1['S/A'].astype(int)

    # Traitement de la colonne 'Dernières performances'
    # Remplacer les parenthèses et leur contenu
    df1['Dernières performances'] = df1['Dernières performances'].str.replace(r'\(\d+\)', '', regex=True)
    df1['Dernières performances'] = df1['Dernières performances'].str.strip()

    # Remplacer les lettres minuscules
    df1['Dernières performances'] = df1['Dernières performances'].str.replace(r'[a-z]', '', regex=True)
    df1['Dernières performances'] = df1['Dernières performances'].str.strip()

    # Remplacer les lettres majuscules par '10'
    df1['Dernières performances'] = df1['Dernières performances'].str.replace(r'[A-Z]', '10', regex=True)
    df1['Dernières performances'] = df1['Dernières performances'].str.strip()

    # Remplacer les nombres supérieurs à 10 par '10'
    df1['Dernières performances'] = df1['Dernières performances'].str.replace(r'\b(\d{2,})\b', '10', regex=True)
    
    return df1


# test d'execution

In [62]:
# Fonction à executer

creer_dossier_course()
date_today()

Dossier créé : scrapping_2025-01-08


'Mercredi 08 Janvier 2025'

In [88]:
driver= webdriver.Chrome()
driver.get("https://www.turfomania.fr/partants-programmes/")
driver = accept_coockies(driver)

links_dict = extract_links(driver)

# On filtre le dict 

root = tk.Tk()
root.title("Gestion des courses")
root.geometry("300x200")

open_popup_button = ttk.Button(root, text="Gérer les courses", command=show_popup)
open_popup_button.pack(expand=True, pady=50)
root.mainloop()

# Scrapper les courses avec le dict filtré

all_dataframes = scrape_links_to_dataframes(driver, links_dict, save_csv=True, show_preview=True)


# Ajouter les 


Erreur lors de l'extraction d'un lien : Message: no such element: Unable to locate element: {"method":"tag name","selector":"a"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF746CE80D5+2992373]
	(No symbol) [0x00007FF74697BFD0]
	(No symbol) [0x00007FF74681590A]
	(No symbol) [0x00007FF74686926E]
	(No symbol) [0x00007FF74686955C]
	(No symbol) [0x00007FF74685C6CC]
	(No symbol) [0x00007FF74688F3AF]
	(No symbol) [0x00007FF74685C596]
	(No symbol) [0x00007FF74688F580]
	(No symbol) [0x00007FF7468AF584]
	(No symbol) [0x00007FF74688F113]
	(No symbol) [0x00007FF74685A918]
	(No symbol) [0x00007FF74685BA81]
	GetHandleVerifier [0x00007FF746D46A2D+3379789]
	GetHandleVerifier [0x00007FF746D5C32D+3468109]
	GetHandleVerifier [0x00007FF746D50043+3418211]
	GetHandleVerifier [0x00007FF746ADC78B+847787]
	(No symbol) [0x00007FF746

C:\Users\Alex\AppData\Local\Temp/ipykernel_10084/2043204429.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Gain'] = df1['Gain'].str.replace('€', '').str.replace(' ', '')
C:\Users\Alex\AppData\Local\Temp/ipykernel_10084/2043204429.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Gain'] = df1['Gain'].str.replace('\xa0', '', regex=True)
C:\Users\Alex\AppData\Local\Temp/ipykernel_10084/2043204429.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

Tableau 2 de PAU :
  N°              Cheval   S/A   Gain Dernières performances
0  1  LONE TIME DU POUEY     4  11960               1 4 8 10
1  2               NAXOS  None   None                   None
2  3     FOREST DU PECOS     4   7150            3 4 9 6 8 2
3  4      EDWARDS DU LIA     4   5980                   10 2
4  5             NOUGARO     4  17020                      1
Tableau sauvegardé : PAU_table2.csv
Tableau 3 de PAU :
  N°          Cheval S/A    Gain        Dernières performances
0  1        MIRALAGO   7  218655   4 1 10 4 10 1 7  2 4 1 10 3
1  2      CALIGRAMME   6  129080  5 10 1 10 2 2 10  1 1 4 1  2
2  3          GALLOX   9  175713    8 7 1  10 6 1 3 3 2 10 6 7
3  4    DOCTOR KALEO  10  248131   2 4  10 1 2 5 1  6 2 7 6 (2
4  5  ECOUTE EN TETE  11  288020      1 8 2 8 4 7 3 8 4  6 4 7
Tableau sauvegardé : PAU_table3.csv
Tableau 4 de PAU :
  N°         Cheval S/A   Gain      Dernières performances
0  1  KANDIDA BELLA   5  32500             5 1 8 1  6 10 4
1  2     

C:\Users\Alex\AppData\Local\Temp/ipykernel_10084/2043204429.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Gain'] = df1['Gain'].str.replace('€', '').str.replace(' ', '')
C:\Users\Alex\AppData\Local\Temp/ipykernel_10084/2043204429.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Gain'] = df1['Gain'].str.replace('\xa0', '', regex=True)
C:\Users\Alex\AppData\Local\Temp/ipykernel_10084/2043204429.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [75]:
links_dict

{'PAU': 'https://www.turfomania.fr/partants-programmes/detail-reunion.php?idreunion=108663&choixtype=1',
 'ANGERS': 'https://www.turfomania.fr/partants-programmes/detail-reunion.php?idreunion=108664&choixtype=1'}

In [84]:
all_dataframes = scrape_links_to_dataframes(driver, links_dict, save_csv=True, show_preview=True)
all_dataframes

Scraping: PAU (https://www.turfomania.fr/partants-programmes/detail-reunion.php?idreunion=108663&choixtype=1)
Erreur lors du chargement de la page : https://www.turfomania.fr/partants-programmes/detail-reunion.php?idreunion=108663&choixtype=1 - Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF746CE80D5+2992373]
	(No symbol) [0x00007FF74697BFD0]
	(No symbol) [0x00007FF74681590A]
	(No symbol) [0x00007FF7467EF4F5]
	(No symbol) [0x00007FF7468963A7]
	(No symbol) [0x00007FF7468AEE72]
	(No symbol) [0x00007FF74688F113]
	(No symbol) [0x00007FF74685A918]
	(No symbol) [0x00007FF74685BA81]
	GetHandleVerifier [0x00007FF746D46A2D+3379789]
	GetHandleVerifier [0x00007FF746D5C32D+3468109]
	GetHandleVerifier [0x00007FF746D50043+3418211]
	GetHandleVerifier [0x00007FF746ADC78B+847787]
	(No symbol) [0x00007FF74698757F]
	(No symbol) [0x00007FF746982FC4]
	(No symbol) [0x00007FF74698315

[]

In [78]:
first_tuple = all_dataframes[0]
first_dataframe = first_tuple[2]
first_dataframe

,N°,Discipline,Nom,,Descriptif,Départ,,PMUPMH
0,Course 1,,PRIX ETALON CHAMAKO - COUPE DES AA - GCH DES 4...,,"Haies - Anglo Arabes - 4 ans - 3500 mètres, Co...",13h11,,PMU
1,Course 2,,PRIX AUGUSTE DE CASTELBAJAC2,,Steeple - Handicap - 5 ans et Plus - 4000 mètr...,13h55,Q+,PMU
2,Course 3,,PRIX JOSEPH BIRABEN3,,Steeple - femelles - Classe 4 - 4 et 5 ans - 3...,14h30,,PMU
3,Course 4,,PRIX SOAL RACING (PRIX D'ETSAUT)4,,Plat - Handicap divise Classe 4 - 5 ans et Plu...,15h08,,PMU
4,Course 5,,PRIX JEAN DE TAURIAC5,,"Haies - Handicap - 5 ans - 3500 mètres, Corde ...",15h43,,PMU
5,Course 6,,PRIX DE LIVRON6,,Plat - Handicap divise Classe 3 - 5 ans et Plu...,16h20,,PMU
6,Course 7,,PRIX EMILE LALANNE7,,Steeple - Handicap - 5 ans et Plus - 4000 mètr...,16h52,,PMU
7,Course 8,,PRIX DE LESCUN8,,Plat - Handicap divise Classe 3 - 5 ans et Plu...,17h25,,PMU


In [80]:
first_dataframe.columns

Index(['N°', 'Discipline', 'Nom', '', 'Descriptif', 'Départ', '', 'PMUPMH'], dtype='object')

In [71]:
links_dict.items()

dict_items([('PAU', 'https://www.turfomania.fr/partants-programmes/detail-reunion.php?idreunion=108663&choixtype=1'), ('ANGERS', 'https://www.turfomania.fr/partants-programmes/detail-reunion.php?idreunion=108664&choixtype=1')])

In [83]:
def scrape_links_to_dataframes(driver, links_dict, save_csv=False, show_preview=False):
    """
    Fonction qui scrape les tableaux HTML à partir d'une liste de liens via Selenium,
    les convertit en DataFrames, et les sauvegarde en CSV si demandé.

    Args:
        driver (WebDriver): Instance Selenium WebDriver.
        links_dict (dict): Dictionnaire contenant les noms des courses comme clés et leurs URLs comme valeurs.
        save_csv (bool): Si True, les DataFrames sont sauvegardés en fichiers CSV. Par défaut False.
        show_preview (bool): Si True, affiche un aperçu des DataFrames. Par défaut False.

    Returns:
        list: Liste de tuples contenant (nom de la course, index du tableau, DataFrame).
    """
    # Liste pour stocker les DataFrames
    all_dataframes = []

    for course_name, url in links_dict.items():
        print(f"Scraping: {course_name} ({url})")
        
        try:
            driver.get(url)
            time.sleep(random.uniform(2, 4))  # Attente aléatoire entre les requêtes

            # Récupérer le HTML de la page
            html = driver.page_source
        except Exception as e:
            print(f"Erreur lors du chargement de la page : {url} - {e}")
            continue

        # Analyser le HTML avec BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')
        
        # Trouver tous les tableaux avec la classe 'tableauLine'
        tables = soup.find_all('table', class_='tableauLine')
        
        # Parcourir les tableaux à partir du deuxième (index 1)
        for table_idx, table in enumerate(tables[1:], start=1):  # Modifier ici pour commencer à partir du 2ème tableau
            # Liste pour stocker les données
            data = []
            headers = []

            # Récupérer les en-têtes de colonnes
            header_row = table.find('thead').find_all('th')
            headers = [th.text.strip() for th in header_row]

            # Récupérer les lignes de données
            rows = table.find('tbody').find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                data.append([cell.text.strip() for cell in cells])
            
            # Créer un DataFrame pour ce tableau
            df = pd.DataFrame(data, columns=headers)
            df = transformer_df(df)
            # Ajouter le DataFrame à la liste
            all_dataframes.append((course_name, table_idx, df))

            # Afficher un aperçu pour vérifier
            if show_preview:
                print(f"Tableau {table_idx + 1} de {course_name} :")
                print(df.head())

            # Sauvegarder le DataFrame en CSV si demandé
            if save_csv:
                date_actuelle = datetime.now().strftime("%Y-%m-%d")
                nom_dossier = f"scrapping_{date_actuelle}"
                chemin_dossier = os.path.join(path_dossier, nom_dossier)
                file_name = f"{course_name.replace(' ', '_')}_table{table_idx + 1}.csv"
                chemin_complet = os.path.join(chemin_dossier, file_name)
                df.to_csv(chemin_complet, index=False)
                print(f"Tableau sauvegardé : {file_name}")

    return all_dataframes


In [74]:
def scrape_links_to_dataframes(driver, links_dict, save_csv=False, show_preview=False):
    """
    Fonction qui scrape les tableaux HTML à partir d'une liste de liens via Selenium,
    les convertit en DataFrames, et les sauvegarde en CSV si demandé.

    Args:
        driver (WebDriver): Instance Selenium WebDriver.
        links_dict (dict): Dictionnaire contenant les noms des courses comme clés et leurs URLs comme valeurs.
        save_csv (bool): Si True, les DataFrames sont sauvegardés en fichiers CSV. Par défaut False.
        show_preview (bool): Si True, affiche un aperçu des DataFrames. Par défaut False.

    Returns:
        list: Liste de tuples contenant (nom de la course, index du tableau, DataFrame).
    """
    # Liste pour stocker les DataFrames
    all_dataframes = []

    for course_name, url in links_dict.items():
        print(f"Scraping: {course_name} ({url})")
        
        try:
            driver.get(url)
            time.sleep(random.uniform(2, 4))


            # Récupérer le HTML de la page
            html = driver.page_source
        except Exception as e:
            print(f"Erreur lors du chargement de la page : {url} - {e}")
            continue

        # Analyser le HTML avec BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')
        
        # Trouver tous les tableaux avec la classe 'tableauLine'
        tables = soup.find_all('table', class_='tableauLine')
        
        # Parcourir chaque tableau trouvé
        for table_idx, table in enumerate(tables):
            # Liste pour stocker les données
            data = []
            headers = []

            # Récupérer les en-têtes de colonnes
            header_row = table.find('thead').find_all('th')
            headers = [th.text.strip() for th in header_row]

            # Récupérer les lignes de données
            rows = table.find('tbody').find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                data.append([cell.text.strip() for cell in cells])
            
            # Créer un DataFrame pour ce tableau
            df = pd.DataFrame(data, columns=headers)
            #df = transformer_df(df)
            # Ajouter le DataFrame à la liste
            all_dataframes.append((course_name, table_idx, df))

            # Afficher un aperçu pour vérifier
            if show_preview:
                print(f"Tableau {table_idx + 1} de {course_name} :")
                print(df.head())

            # Sauvegarder le DataFrame en CSV si demandé
            if save_csv:
                date_actuelle = datetime.now().strftime("%Y-%m-%d")
                nom_dossier = f"scrapping_{date_actuelle}"
                chemin_dossier = os.path.join(path_dossier, nom_dossier)
                file_name = f"{course_name.replace(' ', '_')}_table{table_idx + 1}.csv"
                chemin_complet = os.path.join(chemin_dossier, file_name)
                df.to_csv(chemin_complet, index=False)
                print(f"Tableau sauvegardé : {file_name}")

    return all_dataframes

In [60]:
all_dataframes[0]

('PAU',
 0,
          N° Discipline                                                Nom    \
 0  Course 1             PRIX ETALON CHAMAKO - COUPE DES AA - GCH DES 4...     
 1  Course 2                                  PRIX AUGUSTE DE CASTELBAJAC2     
 2  Course 3                                          PRIX JOSEPH BIRABEN3     
 3  Course 4                             PRIX SOAL RACING (PRIX D'ETSAUT)4     
 4  Course 5                                         PRIX JEAN DE TAURIAC5     
 5  Course 6                                               PRIX DE LIVRON6     
 6  Course 7                                           PRIX EMILE LALANNE7     
 7  Course 8                                               PRIX DE LESCUN8     
 
                                           Descriptif Départ     PMUPMH  
 0  Haies - Anglo Arabes - 4 ans - 3500 mètres, Co...  13h11        PMU  
 1  Steeple - Handicap - 5 ans et Plus - 4000 mètr...  13h55  Q+    PMU  
 2  Steeple - femelles - Classe 4 - 4 et 5 a

# Scrapping deuxieme site 

In [4]:
def accept_coockies2(driver:webdriver.Chrome):
    """ 
    Dans cette fonction je mets un temps au pire entre 5 et 8 secondes, j'attends l'affichage
    du bouton coockie et je click dessus.
    """
    time.sleep(random.uniform(5,8))
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "didomi-notice-agree-button"))).click()    
    return driver


def date_today2():
    return datetime.now().strftime("%Y-%m-%d")

In [5]:
driver= webdriver.Chrome()
driver.get("https://www.geny.com/")
driver = accept_coockies2(driver)
html_content = driver.page_source
time.sleep(random.uniform(2, 4))
driver.quit()
soup = BeautifulSoup(html_content, 'html.parser')

courses_dict = {}
today_date = date_today2()

for row in soup.find_all('tr'):
    link_tag = row.find('a')

    if link_tag:
        course_name = link_tag.text.strip()
        course_link = link_tag['href']

# S'il y a deux fois Vincennes ca garde le dernier donc je filtre sur la date


        if "reunion" in course_link and today_date in course_link:
            course_link = "https://www.geny.com" + course_link if course_link.startswith('/') else course_link
            courses_dict[course_name] = course_link

def filter_dict():
    global courses_dict  # Accès global pour modifier courses_dict
    # Filtrer les courses sélectionnées
    courses_dict = {name: url for name, url in courses_dict.items() if checkboxes[name].get()}
    # Afficher les résultats filtrés dans la console
    print("Courses sélectionnées :")
    for name, url in courses_dict.items():
        print(f"{name}: {url}")
    # Affichage de confirmation
    confirmation_label.config(text=f"{len(courses_dict)} courses sélectionnées.")

# Création de la fenêtre principale
root = tk.Tk()
root.title("Sélection des Réunions")
root.geometry("400x300")

# Titre de l'interface
title_label = tk.Label(root, text="Sélectionnez les réunions à conserver :", font=("Arial", 12))
title_label.pack(pady=10)

# Création d'un conteneur pour les cases à cocher
checkboxes_frame = ttk.Frame(root)
checkboxes_frame.pack(pady=10, fill="x")

# Création des cases à cocher
checkboxes = {}
for name in courses_dict.keys():
    var = tk.BooleanVar()  # Variable associée à chaque case
    checkboxes[name] = var
    checkbox = ttk.Checkbutton(checkboxes_frame, text=name, variable=var)
    checkbox.pack(anchor="w", padx=10)

# Bouton pour valider la sélection
validate_button = ttk.Button(root, text="Valider la sélection", command=filter_dict)
validate_button.pack(pady=10)

# Label pour afficher la confirmation
confirmation_label = tk.Label(root, text="", font=("Arial", 10), fg="green")
confirmation_label.pack(pady=10)

# Lancer l'interface
root.mainloop()

In [12]:
courses_dict

{'Vincennes': 'https://www.geny.com/reunions-courses-pmu/_d2025-01-18?#reunion1',
 'Cagnes-sur-Mer': 'https://www.geny.com/reunions-courses-pmu/_d2025-01-18?#reunion3',
 'Lyon-La Soie': 'https://www.geny.com/reunions-courses-pmu/_d2025-01-18?#reunion4',
 'Toulouse': 'https://www.geny.com/reunions-courses-pmu/_d2025-01-18?#reunion5'}

## Scraper les liens des dicts

To do :


2) AJOUTER LES proprio et mettre dans le df 

3) rendre le programme indetectable

4) améliorer front

